In [3]:
import sqlalchemy as sqla
import pymysql
import pandas as pd
import os

username = "user"  
password = "grad5100user"  
awsresource = "database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"  
port = 3306

In [8]:
dbname = "topChef"
dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)

In [17]:
# Query to filter out records from 'Masters' or 'Canada' seasons from chefdetails
chefdetails_query = """
SELECT *
FROM chefdetails
WHERE season NOT LIKE '%%Masters%%'
AND season NOT LIKE '%%Canada%%'
"""

# Query to filter out records from 'Masters' or 'Canada' seasons from judges
judges_query = """
SELECT *
FROM judges
WHERE season NOT LIKE '%%Masters%%'
AND season NOT LIKE '%%Canada%%'
"""

# Execute the queries
with engine.connect() as conn:
    chefdetails_filtered = pd.read_sql_query(chefdetails_query, conn)
    judges_filtered = pd.read_sql_query(judges_query, conn)

print("Chef Details Filtered:")
chefdetails_filtered.head()

print("Judges Filtered:")
judges_filtered.head()

Chef Details Filtered:
Judges Filtered:


,num,season,seasonNumber,series,episode,challengeType,outcomeType,guestJudge,competedOnTC,otherShows
0,0,All Stars: New York,8,US,1,Quickfire,Team,Tom Colicchio,None,Top Chef Judge
1,1,All Stars: New York,8,US,1,Elimination,Individual,Anthony Bourdain,None,Parts Unknown; No Reservations
2,2,All Stars: New York,8,US,2,Quickfire,Individual,Joe Jonas,None,None
3,3,All Stars: New York,8,US,2,Elimination,Team,Katie Lee,None,Beat Bobby Flay Judge
4,4,All Stars: New York,8,US,3,Quickfire,Team,David Chang,None,None


In [16]:
# Query to select only the columns of interest from the chefdetails table
chefdetails_trimmed_query = """
SELECT season, seasonNumber, name, gender, placement
FROM chefdetails
WHERE season NOT LIKE '%%Masters%%'
AND season NOT LIKE '%%Canada%%'
"""

# Query to select only the columns of interest from the judges table
judges_trimmed_query = """
SELECT season, seasonNumber, episode, challengeType, guestJudge, competedOnTC
FROM judges
WHERE season NOT LIKE '%%Masters%%'
AND season NOT LIKE '%%Canada%%'
"""

# Execute the queries
with engine.connect() as conn:
    chefdetails_trimmed = pd.read_sql_query(chefdetails_trimmed_query, conn)
    judges_trimmed = pd.read_sql_query(judges_trimmed_query, conn)
    
print("Chef Details Trimmed:")
chefdetails_trimmed.head()

print("Judges Trimmed:")
judges_trimmed.head()

Chef Details Trimmed:
Judges Trimmed:


,season,seasonNumber,episode,challengeType,guestJudge,competedOnTC
0,All Stars: New York,8,1,Quickfire,Tom Colicchio,None
1,All Stars: New York,8,1,Elimination,Anthony Bourdain,None
2,All Stars: New York,8,2,Quickfire,Joe Jonas,None
3,All Stars: New York,8,2,Elimination,Katie Lee,None
4,All Stars: New York,8,3,Quickfire,David Chang,None


In [19]:
# Define the query to select 20 unique seasons excluding 'Masters' or 'Canada'
unique_seasons_query = """
SELECT DISTINCT season, seasonNumber
FROM chefdetails
WHERE season NOT LIKE '%%Masters%%' AND season NOT LIKE '%%Canada%%'
ORDER BY seasonNumber
LIMIT 20
"""

# Use SQLAlchemy to connect to the database and execute the query
with engine.connect() as conn:
    unique_seasons = pd.read_sql_query(unique_seasons_query, conn)

# Print the result to see the DataFrame
print("Unique Seasons:")
unique_seasons


Unique Seasons:


,season,seasonNumber
0,San Francisco,1
1,Los Angeles,2
2,Miami,3
3,Chicago,4
4,New York,5
5,Las Vegas,6
6,D.C.,7
7,All Stars: New York,8
8,Texas,9
9,Seattle,10


In [20]:
# 4a. Count of male and female contestants
query_4a = """
SELECT gender, COUNT(*) as count
FROM chefdetails
GROUP BY gender
"""

# 4b. Count of male and female winners (placement==1)
query_4b = """
SELECT gender, COUNT(*) as count
FROM chefdetails
WHERE placement = 1
GROUP BY gender
"""

# 4c. Count of male and female in top 3 finalists (placement=1,2,3)
query_4c = """
SELECT gender, COUNT(*) as count
FROM chefdetails
WHERE placement IN (1, 2, 3)
GROUP BY gender
"""

# Execute the queries and print the results
with engine.connect() as conn:
    # Execute query 4a
    gender_counts = pd.read_sql_query(query_4a, conn)
    print("4a. Count of male and female contestants:")
    print(gender_counts)

    # Execute query 4b
    winners_gender_counts = pd.read_sql_query(query_4b, conn)
    print("\n4b. Count of male and female winners (placement==1):")
    print(winners_gender_counts)

    # Execute query 4c
    top3_gender_counts = pd.read_sql_query(query_4c, conn)
    print("\n4c. Count of male and female in top 3 finalists (placement=1,2,3):")
    print(top3_gender_counts)


4a. Count of male and female contestants:
   gender  count
0    Male    235
1  Female    181

4b. Count of male and female winners (placement==1):
   gender  count
0    Male     20
1  Female      6

4c. Count of male and female in top 3 finalists (placement=1,2,3):
   gender  count
0    Male     50
1  Female     30


In [21]:
# 4d. Compare the number of times a female was placed in the top 3 with the number of times a female won
query_4d_top3 = """
SELECT COUNT(*) as top_3_count
FROM chefdetails
WHERE placement IN (1, 2, 3) AND gender = 'Female'
"""

query_4d_winner = """
SELECT COUNT(*) as winner_count
FROM chefdetails
WHERE placement = 1 AND gender = 'Female'
"""

# Execute the queries and print the results
with engine.connect() as conn:
    # Execute query 4d - top 3
    female_top3_count = pd.read_sql_query(query_4d_top3, conn)
    # Execute query 4d - winners
    female_winner_count = pd.read_sql_query(query_4d_winner, conn)

# Print the results for comparison
print("4d. Number of times a female was in the top 3 and number of times a female won:")
print("Top 3 count:", female_top3_count.iloc[0]['top_3_count'])
print("Winner count:", female_winner_count.iloc[0]['winner_count'])


4d. Number of times a female was in the top 3 and number of times a female won:
Top 3 count: 30
Winner count: 6


In [22]:
# Calculate the proportion
top_3_count = female_top3_count.iloc[0]['top_3_count']
winner_count = female_winner_count.iloc[0]['winner_count']
proportion = winner_count / top_3_count

print("Proportion of top 3 finishes that are wins:", proportion)


Proportion of top 3 finishes that are wins: 0.2


In [28]:
# Query to count the total number of top 3 placements
query_total_top3 = """
SELECT COUNT(*) as top_3_count
FROM chefdetails
WHERE placement IN (1, 2, 3)
"""

# Query to count the total number of wins
query_total_wins = """
SELECT COUNT(*) as winner_count
FROM chefdetails
WHERE placement = 1
"""

# Execute the queries and retrieve the results
with engine.connect() as conn:
    total_top3_count = pd.read_sql_query(query_total_top3, conn)
    total_winner_count = pd.read_sql_query(query_total_wins, conn)


# Print the results
print("Total top 3 placements:", total_top3_count)
print("Total wins:", total_winner_count)

Total top 3 placements:    top_3_count
0           80
Total wins:    winner_count
0            26


In [29]:
# Extract the counts from the DataFrame
total_top3_count = total_top3_count.iloc[0]['top_3_count']
total_winner_count = total_winner_count.iloc[0]['winner_count']

In [30]:
from scipy.stats import chi2_contingency
# create a contingency table for the chi-squared test
contingency_table = [
    [female_winner_count.iloc[0]['winner_count'], female_top3_count.iloc[0]['top_3_count'] - female_winner_count.iloc[0]['winner_count']],
    [total_winner_count - female_winner_count.iloc[0]['winner_count'], total_top3_count - female_top3_count.iloc[0]['top_3_count']]
]

# Perform the chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the results
print(f"Chi-squared: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:")
print(expected)

# Interpret the p-value
alpha = 0.05  # Typical threshold of significance
if p < alpha:
    print("The difference in proportions is statistically significant.")
else:
    print("There is no statistically significant difference in proportions.")


Chi-squared: 0.4182754182754183
P-value: 0.5177988337439523
Degrees of freedom: 1
Expected frequencies:
[[ 7.8 22.2]
 [18.2 51.8]]
There is no statistically significant difference in proportions.


Based on the data provided and the statistical test conducted, there is no evidence to suggest that women who place in the top 3 are less likely than expected to ultimately win the competition.

The chi-squared test compares the observed frequencies of events (in this case, female contestants winning versus being in the top 3) with the frequencies that would be expected if there was no effect or difference. The "Expected frequencies" output shows what the distribution of wins and top 3 placements would look like if there were no difference in the likelihood of winning between those who placed in the top 3.

Since the p-value is high, it indicates that any difference between the observed and expected frequencies is likely due to random chance rather than a systematic effect. Therefore, from a statistical standpoint, women who place in the top 3 are winning as often as would be expected by chance; they are not underperforming or overperforming relative to their presence in the top 3.

In [34]:
# Question 5

# SQL query to create the required table with unique winners
query_winners_guest_judges_unique = """
SELECT DISTINCT 
    c.name as WinnerName, 
    c.season as SeasonWon, 
    CASE 
        WHEN j.guestJudge IS NOT NULL THEN 'Yes' 
        ELSE 'No' 
    END as ServedAsGuestJudge
FROM chefdetails c
LEFT JOIN (
    SELECT DISTINCT guestJudge, season
    FROM judges
) j ON c.name = j.guestJudge AND c.season = j.season
WHERE c.placement = 1
GROUP BY c.name, c.season
"""

# Execute the query and retrieve the results
with engine.connect() as conn:
    winners_guest_judges_unique = pd.read_sql_query(query_winners_guest_judges_unique, conn)

# Print the result
print("Table of First Place Winners and Guest Judge Status (Unique):")
winners_guest_judges_unique


Table of First Place Winners and Guest Judge Status (Unique):


,WinnerName,SeasonWon,ServedAsGuestJudge
0,Richard Blais,All Stars: New York,No
1,Melissa King,All-Stars L.A.,No
2,Mei Lin,Boston,No
3,Jeremy Ford,California,No
4,Ross Larkin,Canada 6,No
5,Brooke Williamson,Charleston,No
6,Stephanie Izard,Chicago,No
7,Kevin Sbraga,D.C.,No
8,Joseph Flamm,Colorado,No
9,Kah-wai Lo,Houston,No


In [36]:
# Question 6

# SQL query to find contestants who were not first place winners but served as a guest judge
query_non_winning_guest_judges = """
SELECT DISTINCT c.name
FROM chefdetails c
INNER JOIN judges j ON c.name = j.guestJudge
WHERE c.placement != 1 OR c.placement IS NULL
"""

# Execute the query and retrieve the results
with engine.connect() as conn:
    non_winning_guest_judges = pd.read_sql_query(query_non_winning_guest_judges, conn)

# Print the result
print("Contestants who were not first place winners but served as a guest judge:")
non_winning_guest_judges


Contestants who were not first place winners but served as a guest judge:


,name
0,Susur Lee
1,Kerry Heffernan
2,Ludo Lefebvre
3,John Besh
4,Lorena Garcia
...,...
57,Miguel Morales
58,Stephen Asprinio
59,Josie Smith-Malave
60,Stefan Richter
